In [22]:
from unittest import mock
from functools import wraps

In [23]:
def get_sum(x, y):
    pass

def wrap_func(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        return 100 + func(*args, **kwargs)
    return wrapper

def mock_func():
    return 30

class MyClass:
    def method1(self):
        return 1000

##### return_value:  

Simulate the return value of a function

##### side_effect:

Simulate the return value of a function. The return value could be a function, an exception or an iterator.

When return_value and side_effect are set at the same time, the result of side_effect will be returned.

In [24]:
get_sum = mock.Mock(name='get_sum', return_value=20, side_effect=mock_func)
result = get_sum()
print(result)
print(get_sum)

30
<Mock name='get_sum' id='2309377346000'>


##### spec:

This can be either a list of strings or an existing object that acts as the specification for the mock object.

In [25]:
my_class = mock.Mock(spec=MyClass())
print(my_class.method1())

<Mock name='mock.method1()' id='2309377346288'>


In [26]:
### The mock object failed to execute when we 
### invoked the method that is in the class.
print(my_class.method2())

AttributeError: Mock object has no attribute 'method2'

##### Mock a function or a class inside a function:

In [ ]:
import datetime
            
def is_weekday():
    today = datetime.datetime.today()
    return (0 <= today.weekday() < 5)

thursday = datetime.datetime(year=2022, month=9, day=15)
### After we assign a mock object to datetime,
### when a function invoke datetime, it is no
### longer the datetime module.
datetime = mock.Mock()
### The method of a mock object is also a mock object.
### we can create the mock object on the fly.
datetime.datetime.today.return_value = thursday 
is_weekday()

True

In [29]:
import datetime
            
def is_weekday():
    today = datetime.datetime.today()
    return (0 <= today.weekday() < 5)

thursday = datetime.datetime(year=2022, month=9, day=15)
### datetime.datetime is written in C which can not be mocked.
datetime.datetime.today = mock.Mock()
datetime.datetime.today.return_value = thursday 
is_weekday()

TypeError: can't set attributes of built-in/extension type 'datetime.datetime'

##### mock.patch:

We can also use patch() to mock objects which looks up an object in a given module and replaces that object with a Mock.

Usually, you use patch() as a decorator or a context manager to provide a scope in which you will mock the tarket object

In [ ]:
import datetime
            
def is_weekday():
    today = datetime.datetime.today()
    return (0 <= today.weekday() < 5)

class FakeDatetime(datetime.datetime):
    """A fake replacement for datetime that can be mocked for testing."""
    @classmethod
    def today(cls):
        return cls(year=2022, month=9, day=15)

### datetime.datetime is written in C which can not be mocked.
### We use a derived class of datetime.datetime, so that the
### derived class is actually datetime.datime except it is
### written in python. 
@mock.patch("datetime.datetime", FakeDatetime)
def test_is_weekday():
    ### datetime.datetime is replaced by FakeDatetime
    ### datetime.datetime.today() is hard coded in class
    ### FakeDatetime. What if we want to test multiple
    ### today() in one testing function.
    print(is_weekday())

test_is_weekday()

##### patch() as a context manager:

Sometimes, you will want to use patch as a context manager rather than a decorator. Some reasons why you might prefer context manager include the following:

* You only want to mock an object for a part of the test scope.

* You are already using too many decorators or parameters, which hurts yout test's readability

In [33]:
import datetime
            
def is_weekday():
    today = datetime.datetime.today()
    return (0 <= today.weekday() < 5)

def test_is_weekday():
    thursday = datetime.datetime(year=2022, month=9, day=15)
    ### datetime.datetime is replaced by a mock object 
    ### inside the context manager. 
    ### We can use module datetime outside the context manager
    ### which is more flexible than using mock.patch() as decorator
    with mock.patch("datetime.datetime") as mock_datetime:
        mock_datetime.today.return_value = thursday
        print(is_weekday())

test_is_weekday()

True
